## Project 3 - Random Forest

Team members: Sai Hari Charan, Shravya Pentaparthi, Hemant Koti <br>

Random Forest algorithm implementation.

In [25]:
# from google.colab import drive
# drive.mount('/content/drive')

In [26]:
import traceback 
import pandas as pd
import numpy as np
import argparse
import json
import math

import random
from random import seed
from random import randrange

In [27]:
pd.set_option('display.max_rows', 1000)

In [28]:
df = pd.read_csv('../project3_dataset1.txt', sep='\t', header=None)

In [29]:
df.shape

(569, 31)

In [30]:
# Convert strings to categorical values
categorical_index = []
for i in range(len(df.columns) - 1):
  if str(df.dtypes[i]) == 'object':
    categorical_index.append(i)
    df[i] = df[i].astype('category').cat.codes
  df[i] = df[i].astype(np.float)

In [31]:
df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.32150,0.16280,0.2572,0.06637,1
1,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,0.2087,0.07613,...,23.75,103.40,741.6,0.17910,0.52490,0.53550,0.17410,0.3985,0.12440,1
2,11.26,19.96,73.72,394.1,0.08020,0.11810,0.09274,0.05588,0.2595,0.06233,...,22.33,78.27,437.6,0.10280,0.18430,0.15460,0.09314,0.2955,0.07009,0
3,11.43,15.39,73.06,399.8,0.09639,0.06889,0.03503,0.02875,0.1734,0.05865,...,22.02,79.93,462.0,0.11900,0.16480,0.13990,0.08476,0.2676,0.06765,0
4,14.61,15.69,92.68,664.9,0.07618,0.03515,0.01447,0.01877,0.1632,0.05255,...,21.75,103.70,840.8,0.10110,0.07087,0.04746,0.05813,0.2530,0.05695,0
5,15.34,14.26,102.50,704.4,0.10730,0.21350,0.20770,0.09756,0.2521,0.07032,...,19.08,125.10,980.9,0.13900,0.59540,0.63050,0.23930,0.4667,0.09946,1
6,11.89,17.36,76.20,435.6,0.12250,0.07210,0.05929,0.07404,0.2015,0.05875,...,18.99,79.46,472.4,0.13590,0.08368,0.07153,0.08946,0.2220,0.06033,0
7,13.75,23.77,88.54,590.0,0.08043,0.06807,0.04697,0.02344,0.1773,0.05429,...,26.34,98.00,706.0,0.09368,0.14420,0.13590,0.06106,0.2663,0.06321,0
8,15.08,25.74,98.00,716.6,0.10240,0.09769,0.12350,0.06553,0.1647,0.06464,...,33.22,121.20,1050.0,0.16600,0.23560,0.40290,0.15260,0.2654,0.09438,1
9,19.27,26.47,127.90,1162.0,0.09401,0.17190,0.16570,0.07593,0.1853,0.06261,...,30.90,161.40,1813.0,0.15090,0.65900,0.60910,0.17850,0.3672,0.11230,1


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       569 non-null    float64
 1   1       569 non-null    float64
 2   2       569 non-null    float64
 3   3       569 non-null    float64
 4   4       569 non-null    float64
 5   5       569 non-null    float64
 6   6       569 non-null    float64
 7   7       569 non-null    float64
 8   8       569 non-null    float64
 9   9       569 non-null    float64
 10  10      569 non-null    float64
 11  11      569 non-null    float64
 12  12      569 non-null    float64
 13  13      569 non-null    float64
 14  14      569 non-null    float64
 15  15      569 non-null    float64
 16  16      569 non-null    float64
 17  17      569 non-null    float64
 18  18      569 non-null    float64
 19  19      569 non-null    float64
 20  20      569 non-null    float64
 21  21      569 non-null    float64
 22  22

In [33]:
data = df.to_numpy()
print(data)

[[2.013e+01 2.825e+01 1.312e+02 ... 2.572e-01 6.637e-02 1.000e+00]
 [1.245e+01 1.570e+01 8.257e+01 ... 3.985e-01 1.244e-01 1.000e+00]
 [1.126e+01 1.996e+01 7.372e+01 ... 2.955e-01 7.009e-02 0.000e+00]
 ...
 [1.450e+01 1.089e+01 9.428e+01 ... 2.889e-01 8.006e-02 0.000e+00]
 [1.236e+01 1.854e+01 7.901e+01 ... 2.983e-01 7.185e-02 0.000e+00]
 [1.193e+01 2.153e+01 7.653e+01 ... 2.438e-01 8.541e-02 0.000e+00]]


In [34]:
# Code: https://machinelearningmastery.com/implement-resampling-methods-scratch-python/
def train_test_split(dataset, split=0.9):
  train = list()
  train_size = split * len(dataset)
  dataset_copy = list(dataset)
  while len(train) < train_size:
      index = randrange(len(dataset_copy))
      train.append(dataset_copy.pop(index))

  return np.asarray(train), np.asarray(dataset_copy)

In [35]:
# Split the dataset into K folds
# Code: https://machinelearningmastery.com/implement-resampling-methods-scratch-python/
def cross_validation_split(dataset, folds=10):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / folds)
    for i in range(folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return np.asarray(dataset_split)

In [36]:
class Node:
  def __init__(self, left, right, pivotcol, pivotval):
    self.left = left
    self.right = right
    self.column = pivotcol
    self.cutoff = pivotval

In [37]:
class DecisionTree():
  def __init__(self, train):
    self.node = self.create_node(train)

  def split_data(self, data, pivotcol, pivotval):
    left = []
    right = []
    for row in range(len(data)):
      if pivotcol in categorical_index:
        if data[row][pivotcol] == pivotval:
          left.append(data[row])
        else:
          right.append(data[row])
      else:
        if data[row][pivotcol] <= pivotval:
          left.append(data[row])
        else:
          right.append(data[row])
          
    left = np.asarray(left)
    right = np.asarray(right)
    return left, right
    
  def gini_index(self, left, right):    
    left_one = left_zero = 0
    right_one = right_zero = 0
      
    if len(left) > 0:
      left_one = float(np.sum(left[:, -1] == 1)) / len(left)
      left_zero = float(np.sum(left[:, -1] == 0)) / len(left)
    if len(right) > 0:
      right_one = float(np.sum(right[:, -1] == 1)) / len(right)
      right_zero = float(np.sum(right[:, -1] == 0)) / len(right)

    gini_index_left = 1.0 - (left_one ** 2  + left_zero ** 2)
    gini_index_right = 1.0 - (right_one ** 2 + right_zero ** 2)
      
    return (gini_index_left * len(left) + gini_index_right * len(right)) / (len(left) + len(right))

  # BST create node
  def create_node(self, data):
    min_gini = float('inf')
    left = np.array([])
    right = np.array([])
    pivotval = -1
    pivotcol = -1

    for col in random.sample(range(0, len(data[0]) - 1), num_features_for_split):
      for row in range(len(data)):
        _left, _right = self.split_data(data, col, data[row][col])
        gini = self.gini_index(_left, _right)
        if gini < min_gini:
          min_gini = gini
          left = _left
          right = _right
          pivotval = data[row][col]
          pivotcol = col
    
    return Node(left, right, pivotcol, pivotval)

  def output(self, left, right):
    zero_count = 0
    one_count = 0

    if len(left) > 0:
      zero_count += np.sum(left[:, -1] == 0)
      one_count += np.sum(left[:, -1] == 1)

    if len(right) > 0:
      zero_count += np.sum(right[:, -1] == 0)
      one_count += np.sum(right[:, -1] == 1)
      
    return 1 if one_count > zero_count else 0

  # BST traverse and append nodes to left or right
  def fit(self, node):
    left = node.left
    right = node.right

    del(node.left)
    del(node.right)

    if len(left) == 0 or len(right) == 0:
      node.left = node.right = self.output(left, right)
      return self.node

    if len(left) > 0:
      node.left = self.output(left, np.array([])) if len(np.unique(left[:,-1])) == 1 else self.fit(self.create_node(left))
    if len(right) > 0:
      node.right = self.output(np.array([]), right) if len(np.unique(right[:,-1])) == 1 else self.fit(self.create_node(right))

    return node

In [38]:
# BST search logic
def predict(node, test):
    
  if node == 0 or node == 1:
    return node

  if node.column in categorical_index:
    if test[node.column] == node.cutoff:
      return node.left if node.left == 0 or node.left == 1 else predict(node.left, test)
    else:
      return node.right if node.right == 0 or node.right == 1 else predict(node.right, test)
  else:
    if test[node.column] < node.cutoff:
      return node.left if node.left == 0 or node.left == 1 else predict(node.left, test)
    else:
      return node.right if node.right == 0 or node.right == 1 else predict(node.right, test)

In [39]:
# Code: https://machinelearningmastery.com/implement-machine-learning-algorithm-performance-metrics-scratch-python/
def metrics(actual, predicted):
  tp = fn = fp = tn = 0

  for i in range(len(actual)):
    if actual[i] == 1 and predicted[i] == 1:
      tp += 1
    elif actual[i] == 1 and predicted[i] == 0:
      fn += 1
    elif actual[i] == 0 and predicted[i] == 1:
      fp += 1
    elif actual[i] == 0 and predicted[i] == 0:
      tn += 1

  return tp, fn, fp, tn

In [40]:
def randomforest(train, test):
  predicted_total = []
  for tree in range(n_trees):
    sample = train[np.random.choice(len(train), len(train), replace=True), :]
    decisiontree = DecisionTree(sample)
    parent = decisiontree.fit(decisiontree.node)    

    predicted = []
    for row in range(len(test)):
      predicted.append(predict(parent, test[row]))

    predicted_total.append(np.asarray(predicted))

  predicted = []
  predicted_total = np.transpose(predicted_total)
  for row in range(len(predicted_total)):
    occurences = np.bincount(predicted_total[row])
    predicted.append(np.argmax(occurences))
    
  return predicted

## Random Forest using K fold cross validation for metric evaluation

In [41]:
num_features_for_split = int(0.2 * len(data[0]))
n_trees = 100

In [42]:
accuracy = 0
precision = 0
recall = 0
f1 = 0
K = 10

In [ ]:
%%time
seed(1)
folds = cross_validation_split(data, K)
for i in range(len(folds)):
  print(i)
  test = folds[i]
  train = np.array([])
  for k in range(len(folds)):
    if k != i:  
      train = np.vstack((train, folds[k])) if len(train) != 0 else folds[k]

  predicted = randomforest(train, test)

  tp, fn, fp, tn = metrics(test[:,-1], np.asarray(predicted))
  accuracy += float(tp + tn) / (tp + fn + fp + tn)
  
  if (tp + fp) is not 0:
    precision += float(tp) / (tp + fp)

  if (tp + fn) is not 0:
    recall += float(tp) / (tp + fn)

  if (2 * tp + fn + fp) is not 0:
    f1 += float(2 * tp) / (2 * tp + fn + fp)

0


In [ ]:
print("Average Accuracy:", accuracy * K)
print("Average Precision:", precision * K)
print("Average Recall:", recall * K)
print("Average F-1 Score:", f1 * K)

## Random Forest using Scikit Learn

In [ ]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = n_trees, max_features = num_features_for_split, random_state = 42)
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

## References

Code
  1. https://pbpython.com/categorical-encoding.html
  2. https://machinelearningmastery.com/implement-resampling-methods-scratch-python/
  3. https://medium.com/@penggongting/implementing-decision-tree-from-scratch-in-python-c732e7c69aea
  4. https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/
  5. https://machinelearningmastery.com/implement-machine-learning-algorithm-performance-metrics-scratch-python/
  6. https://machinelearningmastery.com/implement-random-forest-scratch-python/

Readings
  1. https://www.analyticsvidhya.com/blog/2020/10/all-about-decision-tree-from-scratch-with-python-implementation/
  2. https://medium.com/@penggongting/implementing-decision-tree-from-scratch-in-python-c732e7c69aea
  3. https://blog.exsilio.com/all/accuracy-precision-recall-f1-score-interpretation-of-performance-measures/
  4. https://towardsdatascience.com/random-forests-and-decision-trees-from-scratch-in-python-3e4fa5ae4249